# Function-Based Middleware

This notebook demonstrates how to create middleware using simple functions rather than classes.

## Key Concepts

- **Simplicity**: Function-based middleware is easier to write and understand
- **Decorator Support**: Can be used with `@agent.middleware` decorator
- **Composability**: Multiple function middleware can be chained together

## Prerequisites

- Azure OpenAI endpoint and deployment configured in `.env`
- `agent-framework` package installed
- Azure CLI authentication

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv('../.env')

project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
model_deployment_name = os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME")

print(f"Project Endpoint: {project_endpoint}")
print(f"Model Deployment: {model_deployment_name}")

In [ ]:
from collections.abc import Awaitable, Callable
from random import randint
from typing import Annotated

from agent_framework import FunctionInvocationContext
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential
from pydantic import Field

## Define Tool Function

In [ ]:
def get_weather(
    location: Annotated[str, Field(description="The location to get the weather for.")],
) -> str:
    """Get the weather for a given location."""
    conditions = ["sunny", "cloudy", "rainy", "stormy"]
    return f"The weather in {location} is {conditions[randint(0, 3)]} with a high of {randint(10, 30)}Â°C."

## Function-Based Middleware

Instead of creating a class, we can use a simple async function that follows the middleware signature.

In [ ]:
async def logging_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """Simple function-based middleware to log function invocations."""
    function_name = context.function.name
    print(f"[Middleware] About to call function: {function_name}")
    
    await next(context)
    
    print(f"[Middleware] Function {function_name} completed")

## Example: Using Function-Based Middleware

In [ ]:
async def main():
    """Main function to demonstrate function-based middleware."""
    print("=== Function-based Middleware Example ===\n")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential).create_agent(
            name="WeatherAgent",
            instructions="You are a helpful weather assistant that provides weather information.",
            tools=get_weather,
            middleware=[logging_middleware],
        ) as agent,
    ):
        print("Agent created with function-based middleware\n")
        
        query = "What's the weather in Tokyo?"
        print(f"User: {query}")
        result = await agent.run(query)
        print(f"Agent: {result.text if result.text else 'No response'}")

await main()

## Key Takeaways

- Function-based middleware is simpler than class-based middleware
- Just need an async function with the right signature: `(context, next_middleware) -> result`
- Perfect for simple logging, timing, or transformation tasks
- Can be mixed with class-based middleware

## Next Steps

- **Class-based middleware** (notebook 3) for more complex scenarios
- **Decorator middleware** (notebook 4) for cleaner syntax